# 3. Práctica: Vectores a palabras

### Ejercicio 1

Obtenga la matriz de co-ocurrencia para un corpus en español y realice los siguientes calculos:
- Las probabilidades conjuntas
$$p(w_i,w_j) = \frac{c_{i,j}}{\sum_i \sum_j c_{i,j}}$$
- Las probabilidades marginales
$$p(w_i) = \sum_j p(w_i,w_j)$$
- Positive Point Wise Mutual Information (PPMI):
$$PPMI(w_i,w_j) = \max\{0, \log_2 \frac{p(w_i,w_j)}{p(w_i)p(w_j)}\}$$

In [1]:
import nltk
from nltk.corpus import europarl_raw, stopwords
import numpy as np
import pandas as pd
from itertools import combinations
import string

nltk.download('stopwords')
nltk.download('europarl_raw')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package europarl_raw to /root/nltk_data...
[nltk_data]   Unzipping corpora/europarl_raw.zip.


True

In [2]:
def limpiar_texto(texto: str) -> list[str]:
    """
    Procesa un texto eliminando signos de puntuación, stopwords y convierte a minúsculas.
    """
    sw = set(stopwords.words("spanish"))
    texto = texto.lower().translate(str.maketrans("", "", string.punctuation))
    tokens = texto.split()
    tokens = [t for t in tokens if t not in sw and t.isalnum()]
    return tokens

corpus_raw = europarl_raw.spanish.raw()
oraciones = corpus_raw.split("\n")
tokens_oraciones = [limpiar_texto(oracion) for oracion in oraciones]

In [3]:
def matriz_coocurrencia(oraciones: list[list[str]], vocabulario: dict[str, int]) -> np.ndarray:
    """
    Calcula la matriz de co-ocurrencia para un conjunto de oraciones.
    """
    matriz = np.zeros((len(vocabulario), len(vocabulario)), dtype=np.float32)
    for oracion in oraciones:
        for w1, w2 in combinations(oracion, 2):
            if w1 in vocabulario and w2 in vocabulario:
                i, j = vocabulario[w1], vocabulario[w2]
                matriz[i, j] += 1
                matriz[j, i] += 1  # Simétrica
    return matriz

palabras = sorted(set(palabra for oracion in tokens_oraciones for palabra in oracion))
indice_palabra = {palabra: idx for idx, palabra in enumerate(palabras)}

matriz_cooc = matriz_coocurrencia(tokens_oraciones, indice_palabra)

df_cooc = pd.DataFrame(matriz_cooc, index=palabras, columns=palabras)
df_cooc.sample(5, axis=0).sample(5, axis=1)

,requisitos,fecunda,peregrinaje,adoptar,gobierno
joven,0.0,0.0,0.0,0.0,0.0
dijeran,0.0,0.0,0.0,0.0,0.0
coreper,0.0,0.0,0.0,0.0,0.0
padecer,0.0,0.0,0.0,0.0,0.0
atribuirá,0.0,0.0,0.0,0.0,0.0


In [4]:
def calcular_probabilidades_conjuntas(matriz: np.ndarray) -> np.ndarray:
    total = matriz.sum()
    return matriz / total if total > 0 else matriz

matriz_conjunta = calcular_probabilidades_conjuntas(matriz_cooc)
df_conjunta = pd.DataFrame(matriz_conjunta, index=palabras, columns=palabras)

In [5]:
def calcular_prob_marginales(matriz_conjunta: np.ndarray) -> np.ndarray:
    return matriz_conjunta.sum(axis=1)

prob_marginales = calcular_prob_marginales(matriz_conjunta)
df_marginales = pd.DataFrame(prob_marginales, index=palabras, columns=["p(w)"])

In [6]:
def calcular_ppmi(matriz_conjunta: np.ndarray, prob_marginales: np.ndarray) -> np.ndarray:
    filas, columnas = matriz_conjunta.shape
    matriz_ppmi = np.zeros_like(matriz_conjunta)

    for i in range(filas):
        for j in range(columnas):
            p_ij = matriz_conjunta[i, j]
            if p_ij > 0:
                pmi = np.log2(p_ij / (prob_marginales[i] * prob_marginales[j]))
                matriz_ppmi[i, j] = max(0, pmi)
    return matriz_ppmi

matriz_ppmi = calcular_ppmi(matriz_conjunta, prob_marginales)
df_ppmi = pd.DataFrame(matriz_ppmi, index=palabras, columns=palabras)

In [7]:
print("Co-ocurrencia de sesiones y declaro:", matriz_cooc[indice_palabra["sesiones"], indice_palabra["declaro"]])
print("Prob. conjunta:", matriz_conjunta[indice_palabra["sesiones"], indice_palabra["declaro"]])
print("Prob. marginal de sesiones:", prob_marginales[indice_palabra["sesiones"]])
print("PPMI:", matriz_ppmi[indice_palabra["sesiones"], indice_palabra["declaro"]])

Co-ocurrencia de sesiones y declaro: 9.0
Prob. conjunta: 1.7592467e-06
Prob. marginal de sesiones: 0.00014719031
PPMI: 9.440523


Ejemplo de resultados

- **Co-ocurrencia de "sesiones" y "declaro"**:  
  `9.0` veces

- **Probabilidad conjunta p(wi, wj)**:  
  `1.76e-06`

- **Probabilidad marginal de "sesiones"**:  
  `0.00014719`

- **PPMI(sesiones, declaro)**:  
  `9.44`

- **Co-ocurrencia**: Estas palabras aparecen juntas 9 veces en las oraciones del corpus, indicando una asociación directa.  
- **Probabilidad conjunta**: Aunque la probabilidad es baja, refleja la co-ocurrencia relativa en todo el corpus.  
- **Probabilidad marginal**: El término "sesiones" representa aproximadamente un 0.015% de las palabras del corpus.  
- **PPMI alto**: La puntuación alta de 9.44 en PPMI sugiere que la relación entre estas palabras es más significativa que el azar; esto ocurre cuando la co-ocurrencia observada es mucho mayor que lo esperado por las frecuencias marginales.

### Ejercicio 2

**Comparación de representaciones**

Aplica reducción de dimensionalidad (a 2D) de los vectores de la matríz con PPMI y de los vectores entrenados en español:

- Realiza un plot de 100 vectores aleatorios (que esten tanto en la matríz como en los vectores entrenados)
- Compara los resultados de los plots:
    - ¿Qué representación dirías que captura mejor relaciones semánticas?
    - Realiza un cuadro comparativo de ambos métodos con ventajas/desventajas

In [12]:
# Dimensionality Reduction (PPMI)
pca_ppmi = PCA(n_components=2)
ppmi_2d = pca_ppmi.fit_transform(df_ppmi)
df_ppmi_2d = pd.DataFrame(ppmi_2d, index=palabras, columns=['x', 'y'])

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
import random
from scipy.spatial.distance import euclidean

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject